In [ ]:
import vertexai
from vertexai import agent_engines
from IPython.display import display, Markdown
import os

### Help Function(s)

In [ ]:
def format_grounding_to_markdown(response_dict):
    """
    Converts Vertex AI Grounding/Search response dictionary to a Markdown string.
    """
    md_lines = []
    
    # --- Section 1: Search Queries ---
    if 'web_search_queries' in response_dict:
        md_lines.append("## 🔍 Search Context")
        for query in response_dict['web_search_queries']:
            md_lines.append(f"* *{query}*")
        md_lines.append("\n---\n")

    # --- Section 2: Key Insights (Supports) ---
    md_lines.append("## 💡 Key Insights & Evidence")
    
    chunks = response_dict.get('grounding_chunks', [])
    supports = response_dict.get('grounding_supports', [])

    for support in supports:
        segment = support.get('segment', {})
        text = segment.get('text', '')
        indices = support.get('grounding_chunk_indices', [])
        
        # Format the main text as a blockquote
        md_lines.append(f"> {text}")
        
        # Format the citations (converting 0-based index to 1-based for readability)
        if indices:
            citations = []
            for i in indices:
                # specific_chunk = chunks[i] # Access full chunk data if needed
                citations.append(f"[{i + 1}]")
            
            md_lines.append(f">\n> **Sources:** {', '.join(citations)}\n")

    # --- Section 3: Reference List ---
    md_lines.append("### 📚 Reference List")
    for idx, chunk in enumerate(chunks):
        web = chunk.get('web', {})
        title = web.get('title', 'Unknown Title')
        uri = web.get('uri', '#')
        domain = web.get('domain', '')
        
        # format: 1. [Title](Link) - Domain
        md_lines.append(f"{idx + 1}. [{title}]({uri}) - *{domain}*")

    return "\n".join(md_lines)



### Main Code

In [ ]:
PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT')

In [ ]:
vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
# List all agents
for agent in agent_engines.list():
    print(f"Name: {agent.display_name}")
    print(f"Resource Name: {agent.resource_name}") # Contains the ID you need
    print("-" * 20)

In [ ]:
remote_app = agent_engines.get("projects/633472233130/locations/us-central1/reasoningEngines/9047199487759482880")

In [ ]:
remote_session = await remote_app.async_create_session(user_id="u_456")
print(remote_session)

In [ ]:
response = []

async for event in remote_app.async_stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="latest innovations in protein bars",
):
    print(event)
    response.append(event)

In [ ]:
display(Markdown(response[1]['content']['parts'][0]['text']))

In [ ]:
markdown_output = format_grounding_to_markdown(response[0]['grounding_metadata'])
display(Markdown(markdown_output))